## Deliverable 3. Create a Travel Itinerary Map.

In [43]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import config

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [44]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Rikitea,PF,79.79,light rain,-23.1203,-134.9692,People ThankYou
1,3,Tessalit,ML,84.70,broken clouds,20.1986,1.0114,NaN
2,5,Vaini,TO,80.80,light rain,-21.2000,-175.2000,Keleti Beach Resort
3,14,Mayumba,GA,79.41,broken clouds,-3.4320,10.6554,LIKWALE LODGE
4,17,Hilo,US,77.25,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel


In [45]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,-23.1203,-134.9692
1,20.1986,1.0114
2,-21.2000,-175.2000
3,-3.4320,10.6554
4,19.7297,-155.0900
...,...,...
221,-9.6567,120.2641
222,1.1667,108.9667
223,-11.0964,-77.6139
224,29.0500,-109.2333


In [46]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [48]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
vacation_start = vacation_df.loc[vacation_df.City == "Ayotzintepec"] 
vacation_end = vacation_df.loc[vacation_df.City == "Ayotzintepec"] 
vacation_stop1 = vacation_df.loc[vacation_df.City == "Acapulco"]
vacation_stop2 = vacation_df.loc[vacation_df.City == "La Orilla"]
vacation_stop3 = vacation_df.loc[vacation_df.City == "Juventino Rosas"]

vacation_start


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
137,438,Ayotzintepec,MX,88.74,few clouds,17.6667,-96.1333,"Hotel ""Punta del Cerro"""


In [49]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[["Lat", "Lng"]].to_numpy()[0]
end = vacation_end[["Lat", "Lng"]].to_numpy()[0]
stop1 = vacation_stop1[["Lat", "Lng"]].to_numpy()[0]
stop2 = vacation_stop2[["Lat", "Lng"]].to_numpy()[0]
stop3 = vacation_stop3[["Lat", "Lng"]].to_numpy()[0]


In [51]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
vacation_itinerary = gmaps.directions_layer(start, end,
        waypoints = [stop1, stop2, stop3], travel_mode = "DRIVING")
fig.add_layer(vacation_itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [52]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_end, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,438,Ayotzintepec,MX,88.74,few clouds,17.6667,-96.1333,"Hotel ""Punta del Cerro"""
1,438,Ayotzintepec,MX,88.74,few clouds,17.6667,-96.1333,"Hotel ""Punta del Cerro"""
2,500,Acapulco,MX,82.22,clear sky,16.8634,-99.8901,HS Hotsson Smart Acapulco
3,617,La Orilla,MX,84.43,clear sky,17.9833,-102.2333,Baymont by Wyndham Lazaro Cardenas
4,653,Juventino Rosas,MX,81.68,clear sky,20.6500,-101.0000,Hotel Plaza


In [53]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [54]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))